In [1]:
import threading
import time
import subprocess
import os
import re
import psutil
import torch
import tempfile
import tf2onnx
import tensorflow as tf
import coremltools as ct
import coremltools as ct
import onnx
import numpy as np
import onnxruntime as ort

from tensorflow.keras.datasets import mnist
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

2024-08-08 22:17:40.571141: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
scikit-learn version 1.3.2 is not supported. Minimum required version: 0.17. Maximum required version: 1.1.2. Disabling scikit-learn conversion API.
XGBoost version 2.1.0 has not been tested with coremltools. You may run into unexpected errors. XGBoost 1.4.2 is the most recent version that has been tested.
Torch version 2.2.2 has not been tested with coremltools. You may run into unexpected errors. Torch 2.2.0 is the most recent version that has been tested.
/opt/anaconda3/envs/env_name/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/sta

In [2]:
# 加载 MNIST 数据集
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# 调整输入数据的形状和范围
X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255
# 将标签转换为 one-hot 编码
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [ ]:

# 创建 CNN 模型
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

# 编译模型
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# 训练模型
model.fit(X_train, y_train, batch_size=128, epochs=2, verbose=1, validation_data=(X_test, y_test))
# 评估模型
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

#model.save('./mnist_models/mnist_cnn_model.h5')
model.save('./models_train/mnist_cnn_model.h5')

### CNN h5

In [2]:
thread_output = {}
duration = 0
# 定义监控保存模型时的资源使用率的线程函数
def monitor_resources_during_save(stop_event):
    cpu_usage = []
    gpu_usage = []

    while not stop_event.is_set():
        cpu_usage.append(psutil.cpu_percent(interval=0.1))
        try:
            gpu_output = subprocess.check_output(['nvidia-smi', 
                                                  '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
            gpu_usage.append(int(gpu_output.strip()))
        except Exception as e:
            gpu_usage.append(None)  # 如果没有GPU或nvidia-smi命令失败，则记录None
        # time.sleep(0.1)

    # 保存监测结果
    thread_output['cpu_usage'] = cpu_usage
    thread_output['gpu_usage'] = gpu_usage
    print("Resource monitoring finished.")

# 定义保存模型的函数
def save_model(stop_event, model):
    global duration
    global inference_duration
    print("Saving model...")
    
    # 测量保存模型的时间
    start_time = time.time()
    # 保存 Keras 模型为 HDF5 格式
    for i in range(0, 1):
        model.save("mnist_cnn_model.h5")
    end_time = time.time()
    duration = end_time - start_time
    print(f'Time taken to save TensorFlow SavedModel: {duration} seconds')

    # 测量推理时间
    X_test_sample = X_test[:100]  # 选择前10个样本进行推理batch_size=10000
    start_time_inference = time.time()
    n_time = 1 # 计数器
    for i in range(n_time): # 重复操作n遍
        model.predict(X_test_sample)
    end_time_inference = time.time()
    inference_duration = end_time_inference - start_time_inference
    print(f'Time taken for inference on {len(X_test_sample)} samples {n_time} times: {inference_duration:.4f} seconds')

    stop_event.set()  # 触发停止其他线程

# 运行外部脚本并捕获输出
def run_script(stop_event):
    try:
        # 使用临时文件来存储子进程输出
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            process = subprocess.Popen(['/Users/anelloyi/Desktop/run_powermetrics.sh'], 
                                       stdout=tmp_file, stderr=subprocess.STDOUT, text=True)
            print("Subprocess started.")
            while not stop_event.is_set():
                if process.poll() is not None:  # 检查进程是否已经结束
                    break
                # time.sleep(0.1)
            
            if process.poll() is None:
                # 尝试终止进程
                process.terminate()
                try:
                    process.wait(timeout=0.1)
                except subprocess.TimeoutExpired:
                    process.kill()  # 如果超时，则强制终止
                    process.wait()
        
        # 读取临时文件的内容
        with open(tmp_file.name, 'r') as f:
            thread_output['powermetrics'] = f.read()
        
        os.remove(tmp_file.name)  # 删除临时文件
        print("Subprocess finished.")
    except Exception as e:
        thread_output['powermetrics'] = str(e)
        print("Exception in subprocess:", str(e))

# 载入数据和模型
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

stop_event = threading.Event()
model = tf.keras.models.load_model("./mnist_models/mnist_cnn_model.h5")

# 创建和启动线程
thread1 = threading.Thread(target=save_model, args=(stop_event, model))
thread2 = threading.Thread(target=monitor_resources_during_save, args=(stop_event,))
thread3 = threading.Thread(target=run_script, args=(stop_event,))
thread3.start()
thread1.start()
thread2.start()
# 等待线程完成
thread1.join()
thread2.join()
thread3.join()

# 输出从线程收集的数据
content = thread_output.get('powermetrics', 'No output captured')
# 将内容按行拆分
lines = content.split('\n')
# 筛选出以 "CPU consume" 开头的行
filtered_lines = [line for line in lines if line.startswith('GPU Power:') or 
                  line.startswith('CPU Power:')] # or
                  # line.startswith('Combined Power (CPU + GPU + ANE):')]
# 将筛选后的行合并为一个字符串，每行之间用换行符分隔
filtered_content = '\n'.join(filtered_lines)

# ----------------------------------------------------------------------------
file_name = f'./mnist_models/output_cnn/output-mnist-cnn-h5.txt'
# ----------------------------------------------------------------------------
with open(file_name, 'w') as file:
    file.write(filtered_content)
    file.write(f'\nTotal Duration(s): {duration:.2f}')
    file.write(f'\nInference Duration(s): {inference_duration:.4f}')
print(f"Content saved to {file_name}")

filtered_lines_count = len(filtered_lines)
print(filtered_lines_count)
print(filtered_lines)

# 提取每一个采样点的数字，即CPU和GPU的具体mV
numbers = []
for line in filtered_lines:
    match = re.search(r'[\d.]+', line)
    if match:
        numbers.append(float(match.group())) 

delta_time = duration * 2 / filtered_lines_count
# print(delta_time)

numbers_scaled = [num * delta_time for num in numbers]
total_energy_consumption = sum(numbers_scaled)
print(f"Total energy consumption: {total_energy_consumption} mV")

Saving model...
Subprocess started.
Time taken to save TensorFlow SavedModel: 0.2993950843811035 seconds
4/4 [==============================] - 0s 5ms/step
Time taken for inference on 100 samples 1 times: 0.3005 seconds
Subprocess finished.
Resource monitoring finished.
Content saved to ./mnist_models/output_cnn/output-mnist-cnn-h5.txt
0
[]


ZeroDivisionError: float division by zero

In [8]:
import time
import threading
import psutil
import subprocess
import tempfile
import os
import re
import tensorflow as tf
from tensorflow.keras.datasets import mnist

thread_output = {}
duration = 0
inference_duration = 0

# 定义监控保存模型时的资源使用率的线程函数
def monitor_resources_during_save(stop_event):
    cpu_usage = []
    gpu_usage = []

    while not stop_event.is_set():
        cpu_usage.append(psutil.cpu_percent(interval=0.1))
        try:
            gpu_output = subprocess.check_output(['nvidia-smi', 
                                                  '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
            gpu_usage.append(int(gpu_output.strip()))
        except Exception as e:
            gpu_usage.append(None)  # 如果没有GPU或nvidia-smi命令失败，则记录None

    # 保存监测结果
    thread_output['cpu_usage'] = cpu_usage
    thread_output['gpu_usage'] = gpu_usage
    print("Resource monitoring finished.")

# 定义保存模型的函数
def save_model(stop_event, model):
    global duration
    global inference_duration
    print("Saving model...")
    
    # 测量保存模型的时间
    start_time = time.time()
    # 保存 Keras 模型为 HDF5 格式
    for i in range(1):  # 修改为2次保存，以提高可靠性
        model.save("mnist_cnn_model.h5")
    end_time = time.time()
    duration = end_time - start_time
    print(f'Time taken to save TensorFlow SavedModel: {duration} seconds')

    # 测量推理时间
    X_test_sample = X_test[:10000]  # 选择前10000个样本进行推理
    batch_size = 128 # 批量大小 32 16 8 4 
    num_batches = len(X_test_sample) // batch_size  # 计算批次数

    start_time_inference = time.time()

    # 批量推理
    for _ in range(1):  # 重复1次进行批量推理
        for batch_idx in range(num_batches):
            batch_start = batch_idx * batch_size
            batch_end = (batch_idx + 1) * batch_size
            batch = X_test_sample[batch_start:batch_end]
            
            # 扩展批次数据的维度
            batch_reshaped = batch
            
            model.predict(batch_reshaped)
    
    end_time_inference = time.time()
    inference_duration = end_time_inference - start_time_inference
    print(f'Time taken for inference on {len(X_test_sample)} samples {1} times: {inference_duration:.4f} seconds')

    stop_event.set()  # 触发停止其他线程

# 运行外部脚本并捕获输出
def run_script(stop_event):
    try:
        # 使用临时文件来存储子进程输出
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            process = subprocess.Popen(['/Users/anelloyi/Desktop/run_powermetrics.sh'], 
                                       stdout=tmp_file, stderr=subprocess.STDOUT, text=True)
            print("Subprocess started.")
            while not stop_event.is_set():
                if process.poll() is not None:  # 检查进程是否已经结束
                    break
            
            if process.poll() is None:
                # 尝试终止进程
                process.terminate()
                try:
                    process.wait(timeout=0.1)
                except subprocess.TimeoutExpired:
                    process.kill()  # 如果超时，则强制终止
                    process.wait()
        
        # 读取临时文件的内容
        with open(tmp_file.name, 'r') as f:
            thread_output['powermetrics'] = f.read()
        
        os.remove(tmp_file.name)  # 删除临时文件
        print("Subprocess finished.")
    except Exception as e:
        thread_output['powermetrics'] = str(e)
        print("Exception in subprocess:", str(e))

# # 载入数据和模型
# (X_train, y_train), (X_test, y_test) = mnist.load_data()
# X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
# X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))
# X_train = X_train.astype('float32') / 255
# X_test = X_test.astype('float32') / 255
# y_train = tf.keras.utils.to_categorical(y_train, 10)
# y_test = tf.keras.utils.to_categorical(y_test, 10)

stop_event = threading.Event()
model = tf.keras.models.load_model("./mnist_models/mnist_cnn_model.h5")

# 创建和启动线程
thread1 = threading.Thread(target=save_model, args=(stop_event, model))
thread2 = threading.Thread(target=monitor_resources_during_save, args=(stop_event,))
thread3 = threading.Thread(target=run_script, args=(stop_event,))
thread1.start()
thread2.start()
thread3.start()

# 等待线程完成
thread1.join()
thread2.join()
thread3.join()

# 输出从线程收集的数据
content = thread_output.get('powermetrics', 'No output captured')
# 将内容按行拆分
lines = content.split('\n')
# 筛选出以 "GPU Power:" 或 "CPU Power:" 开头的行
filtered_lines = [line for line in lines if line.startswith('GPU Power:') or 
                  line.startswith('CPU Power:')] 

# 将筛选后的行合并为一个字符串，每行之间用换行符分隔
filtered_content = '\n'.join(filtered_lines)

# ----------------------------------------------------------------------------
file_name = f'./mnist_models/output_cnn/output-mnist-cnn-h5.txt'
# ----------------------------------------------------------------------------
with open(file_name, 'w') as file:
    file.write(filtered_content)
    file.write(f'\nTotal Duration(s): {duration:.2f}')
    file.write(f'\nInference Duration(s): {inference_duration:.4f}')
print(f"Content saved to {file_name}")

filtered_lines_count = len(filtered_lines)
#print(filtered_lines_count)
#print(filtered_lines)

duration = inference_duration
# 提取每一个采样点的数字，即CPU和GPU的具体mV
numbers = []
for line in filtered_lines:
    match = re.search(r'[\d.]+', line)
    if match:
        numbers.append(float(match.group())) 

delta_time = duration * 2 / filtered_lines_count

numbers_scaled = [num * delta_time for num in numbers]
total_energy_consumption = sum(numbers_scaled)
print(f"Total energy consumption: {total_energy_consumption} mV")

Saving model...
Subprocess started.
Time taken to save TensorFlow SavedModel: 0.05698871612548828 seconds
4/4 [==============================] - 0s 4ms/step
Time taken for inference on 10000 samples 1 times: 7.1372 seconds
Subprocess finished.
Resource monitoring finished.
Content saved to ./mnist_models/output_cnn/output-mnist-cnn-h5.txt
Total energy consumption: 65512.37684197979 mV


### cnn H5

In [3]:
import time
import threading
import psutil
import subprocess
import tempfile
import os
import re
import tensorflow as tf
from tensorflow.keras.datasets import mnist

thread_output = {}
duration = 0
inference_duration = 0

# 定义监控保存模型时的资源使用率的线程函数
def monitor_resources_during_save(stop_event):
    cpu_usage = []
    gpu_usage = []

    while not stop_event.is_set():
        cpu_usage.append(psutil.cpu_percent(interval=0.1))
        try:
            gpu_output = subprocess.check_output(['nvidia-smi', 
                                                  '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
            gpu_usage.append(int(gpu_output.strip()))
        except Exception as e:
            gpu_usage.append(None)  # 如果没有GPU或nvidia-smi命令失败，则记录None

    # 保存监测结果
    thread_output['cpu_usage'] = cpu_usage
    thread_output['gpu_usage'] = gpu_usage
    print("Resource monitoring finished.")

# 定义保存模型的函数
def save_model(stop_event, model):
    global duration
    global inference_duration
    print("Saving model...")
    
    # 测量保存模型的时间
    start_time = time.time()
    # 保存 Keras 模型为 HDF5 格式
    for i in range(1):  # 修改为2次保存，以提高可靠性
        model.save("mnist_cnn_model.h5")
    end_time = time.time()
    duration = end_time - start_time
    print(f'Time taken to save TensorFlow SavedModel: {duration} seconds')

    # 测量推理时间
    X_test_sample = X_test[:1000]  # 选择前10000个样本进行推理
    batch_size = 1  # 批量大小
    num_batches = len(X_test_sample) // batch_size  # 计算批次数

    start_time_inference = time.time()

    # 批量推理
    for _ in range(1):  # 重复32次进行批量推理
        for batch_idx in range(num_batches):
            batch_start = batch_idx * batch_size
            batch_end = (batch_idx + 1) * batch_size
            batch = X_test_sample[batch_start:batch_end]
            
            # 扩展批次数据的维度
            batch_reshaped = batch
            
            model.predict(batch_reshaped)
    
    end_time_inference = time.time()
    inference_duration = end_time_inference - start_time_inference
    print(f'Time taken for inference on {len(X_test_sample)} samples {1} times: {inference_duration:.4f} seconds')

    stop_event.set()  # 触发停止其他线程

# 运行外部脚本并捕获输出
def run_script(stop_event):
    try:
        # 使用临时文件来存储子进程输出
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            process = subprocess.Popen(['/Users/anelloyi/Desktop/run_powermetrics.sh'], 
                                       stdout=tmp_file, stderr=subprocess.STDOUT, text=True)
            print("Subprocess started.")
            while not stop_event.is_set():
                if process.poll() is not None:  # 检查进程是否已经结束
                    break
            
            if process.poll() is None:
                # 尝试终止进程
                process.terminate()
                try:
                    process.wait(timeout=0.1)
                except subprocess.TimeoutExpired:
                    process.kill()  # 如果超时，则强制终止
                    process.wait()
        
        # 读取临时文件的内容
        with open(tmp_file.name, 'r') as f:
            thread_output['powermetrics'] = f.read()
        
        os.remove(tmp_file.name)  # 删除临时文件
        print("Subprocess finished.")
    except Exception as e:
        thread_output['powermetrics'] = str(e)
        print("Exception in subprocess:", str(e))

# # 载入数据和模型
# (X_train, y_train), (X_test, y_test) = mnist.load_data()
# X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
# X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))
# X_train = X_train.astype('float32') / 255
# X_test = X_test.astype('float32') / 255
# y_train = tf.keras.utils.to_categorical(y_train, 10)
# y_test = tf.keras.utils.to_categorical(y_test, 10)

stop_event = threading.Event()
model = tf.keras.models.load_model("./mnist_models/mnist_cnn_model.h5")

# 创建和启动线程
thread1 = threading.Thread(target=save_model, args=(stop_event, model))
thread2 = threading.Thread(target=monitor_resources_during_save, args=(stop_event,))
thread3 = threading.Thread(target=run_script, args=(stop_event,))
thread1.start()
thread2.start()
thread3.start()

# 等待线程完成
thread1.join()
thread2.join()
thread3.join()

# 输出从线程收集的数据
content = thread_output.get('powermetrics', 'No output captured')
# 将内容按行拆分
lines = content.split('\n')
# 筛选出以 "GPU Power:" 或 "CPU Power:" 开头的行
filtered_lines = [line for line in lines if line.startswith('GPU Power:') or 
                  line.startswith('CPU Power:')] 

# 将筛选后的行合并为一个字符串，每行之间用换行符分隔
filtered_content = '\n'.join(filtered_lines)

# ----------------------------------------------------------------------------
file_name = f'./mnist_models/output_cnn/output-mnist-cnn-h5.txt'
# ----------------------------------------------------------------------------
with open(file_name, 'w') as file:
    file.write(filtered_content)
    file.write(f'\nTotal Duration(s): {duration:.2f}')
    file.write(f'\nInference Duration(s): {inference_duration:.4f}')
print(f"Content saved to {file_name}")

filtered_lines_count = len(filtered_lines)
# print(filtered_lines_count)
# print(filtered_lines)

duration = inference_duration
# 提取每一个采样点的数字，即CPU和GPU的具体mV
numbers = []
for line in filtered_lines:
    match = re.search(r'[\d.]+', line)
    if match:
        numbers.append(float(match.group())) 

delta_time = duration * 2 / filtered_lines_count

numbers_scaled = [num * delta_time for num in numbers]
total_energy_consumption = sum(numbers_scaled)
print(f"Total energy consumption: {total_energy_consumption} mV")


Saving model...
Subprocess started.
Time taken to save TensorFlow SavedModel: 0.1022939682006836 seconds
1/1 [==============================] - 0s 24ms/step
Time taken for inference on 1000 samples 1 times: 69.7560 seconds
Subprocess finished.
Resource monitoring finished.
Content saved to ./mnist_models/output_cnn/output-mnist-cnn-h5.txt
Total energy consumption: 633522.4598342732 mV


### CNN ONNX

In [9]:
import time
import numpy as np
import onnxruntime as ort
import tf2onnx
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import threading
import subprocess
import tempfile
import os
import psutil
import re

# 全局变量
thread_output = {}
duration = 0
inference_duration = 0

# 定义监控保存模型时的资源使用率的线程函数
def monitor_resources_during_save(stop_event):
    cpu_usage = []
    gpu_usage = []

    while not stop_event.is_set():
        cpu_usage.append(psutil.cpu_percent(interval=0.1))
        try:
            gpu_output = subprocess.check_output(['nvidia-smi', 
                                                  '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
            gpu_usage.append(int(gpu_output.strip()))
        except Exception as e:
            gpu_usage.append(None)  # 如果没有GPU或nvidia-smi命令失败，则记录None

    # 保存监测结果
    thread_output['cpu_usage'] = cpu_usage
    thread_output['gpu_usage'] = gpu_usage
    print("Resource monitoring finished.")

# 定义保存模型的函数
def save_model(stop_event, model):
    global duration
    global inference_duration
    global X_test  # 确保引用全局变量

    print("Saving model...")
    # --------------------------------------------------------------------------------
    start_time = time.time()
    # 使用 tf2onnx 保存模型为ONNX格式
    spec = (tf.TensorSpec((None, 28, 28, 1), tf.float32, name="input"),)
    output_path = "mnist_cnn_model.onnx"
    
    for _ in range(5):  # 保存 ONNX 模型的循环次数为 5
        model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, output_path=output_path)

    end_time = time.time()
    duration = end_time - start_time
    print(f'Time taken to save ONNX model: {duration} seconds')
    # --------------------------------------------------------------------------------

    # 测量推理时间
    # 使用 ONNX Runtime 加载模型
    ort_session = ort.InferenceSession(output_path)
    
    # 准备 ONNX 输入
    X_test_sample = X_test[:10000].astype(np.float32)  # 选择前5000个样本进行推理
    
    # 测量推理时间
    start_time_inference = time.time()
    
    # 获取模型输入名称
    input_name = ort_session.get_inputs()[0].name
    
    # 批量推理
    batch_size = 128
    num_batches = len(X_test_sample) // batch_size
    for _ in range(1):  # 推理循环次数为 1
        for batch_idx in range(num_batches):
            batch_start = batch_idx * batch_size
            batch_end = batch_start + batch_size
            batch = X_test_sample[batch_start:batch_end]
            ort_outs = ort_session.run(None, {input_name: batch})
    
    end_time_inference = time.time()
    inference_duration = end_time_inference - start_time_inference
    print(f'Time taken for inference on {len(X_test_sample)} samples: {inference_duration:.4f} seconds')

    stop_event.set()  # 触发停止其他线程

# 运行外部脚本并捕获输出
def run_script(stop_event):
    try:
        # 使用临时文件来存储子进程输出
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            process = subprocess.Popen(['/Users/anelloyi/Desktop/run_powermetrics.sh'], 
                                       stdout=tmp_file, stderr=subprocess.STDOUT, text=True)
            print("Subprocess started.")
            while not stop_event.is_set():
                if process.poll() is not None:  # 检查进程是否已经结束
                    break
            
            if process.poll() is None:
                # 尝试终止进程
                process.terminate()
                try:
                    process.wait(timeout=0.1)
                except subprocess.TimeoutExpired:
                    process.kill()  # 如果超时，则强制终止
                    process.wait()
        
        # 读取临时文件的内容
        with open(tmp_file.name, 'r') as f:
            thread_output['powermetrics'] = f.read()
        
        os.remove(tmp_file.name)  # 删除临时文件
        print("Subprocess finished.")
    except Exception as e:
        thread_output['powermetrics'] = str(e)
        print("Exception in subprocess:", str(e))

# 载入数据和模型
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

stop_event = threading.Event()
model = tf.keras.models.load_model("./mnist_models/mnist_cnn_model.h5")

# 创建和启动线程
thread1 = threading.Thread(target=save_model, args=(stop_event, model))
thread2 = threading.Thread(target=monitor_resources_during_save, args=(stop_event,))
thread3 = threading.Thread(target=run_script, args=(stop_event,))
thread3.start()
thread1.start()
thread2.start()
# 等待线程完成
thread1.join()
thread2.join()
thread3.join()

# 输出从线程收集的数据
content = thread_output.get('powermetrics', 'No output captured')
# 将内容按行拆分
lines = content.split('\n')
# 筛选出以 "CPU consume" 开头的行
filtered_lines = [line for line in lines if line.startswith('GPU Power:') or 
                  line.startswith('CPU Power:')] 

# 将筛选后的行合并为一个字符串，每行之间用换行符分隔
filtered_content = '\n'.join(filtered_lines)

# ----------------------------------------------------------------------------
file_name = f'./mnist_models/output_cnn/output-mnist-cnn-onnx.txt'
# ----------------------------------------------------------------------------
with open(file_name, 'w') as file:
    file.write(filtered_content)
    file.write(f'\nTotal Duration(s): {duration:.2f}')
    file.write(f'\nInference Duration(s): {inference_duration:.4f}')
print(f"Content saved to {file_name}")

filtered_lines_count = len(filtered_lines)
# print(filtered_lines_count)
# print(filtered_lines)

duration = inference_duration
# 提取每一个采样点的数字，即CPU和GPU的具体mV
numbers = []
for line in filtered_lines:
    match = re.search(r'[\d.]+', line)
    if match:
        numbers.append(float(match.group())) 

delta_time = duration * 2 / filtered_lines_count

numbers_scaled = [num * delta_time for num in numbers]
total_energy_consumption = sum(numbers_scaled)
print(f"Total energy consumption: {total_energy_consumption} mV")


Saving model...
Subprocess started.
Time taken to save ONNX model: 1.1493628025054932 seconds
Time taken for inference on 10000 samples: 0.3822 seconds
Subprocess finished.
Resource monitoring finished.
Content saved to ./mnist_models/output_cnn/output-mnist-cnn-onnx.txt
Total energy consumption: 4980.994953481356 mV


### CNN core ml

In [6]:
thread_output = {}
duration = 0
inference_duration = 0  # 添加推理时间变量

# 定义监控保存模型时的资源使用率的线程函数
def monitor_resources_during_save(stop_event):
    cpu_usage = []
    gpu_usage = []

    while not stop_event.is_set():
        cpu_usage.append(psutil.cpu_percent(interval=0.1))
        try:
            gpu_output = subprocess.check_output(['nvidia-smi', 
                                                  '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
            gpu_usage.append(int(gpu_output.strip()))
        except Exception as e:
            gpu_usage.append(None)  # 如果没有GPU或nvidia-smi命令失败，则记录None

    # 保存监测结果
    thread_output['cpu_usage'] = cpu_usage
    thread_output['gpu_usage'] = gpu_usage
    print("Resource monitoring finished.")

# 定义保存模型的函数
def save_model(stop_event, model):
    global duration
    global inference_duration
    print("Saving model...")
    start_time = time.time()

    # 将 TensorFlow Keras 模型转换为 Core ML 模型
    for i in range(5):
        try:
            input_shape = ct.Shape(shape=(1, 28, 28, 1))
            input_feature = ct.TensorType(name="conv2d_input", shape=input_shape)
            mlmodel = ct.convert(model, inputs=[input_feature])
            mlmodel.save(f"mnist_cnn_model_coreML.mlpackage")
        except Exception as e:
            print(f"Error saving model {i}: {e}")

    end_time = time.time()
    duration = end_time - start_time
    print(f'Time taken to save Core ML model: {duration} seconds')

    
    
    # 测量推理时间
    X_test_sample = X_test[:10000].astype(np.float32)  # 选择前100个样本进行推理
    start_time_inference = time.time()

    batch_size =  1 # 批量大小
    num_batches = len(X_test_sample) // batch_size  # 计算批次数
    predictions = []

    # 进行批量推理
    for batch_idx in range(num_batches):
        batch_start = batch_idx * batch_size
        batch_end = (batch_idx + 1) * batch_size
        batch = X_test_sample[batch_start:batch_end]

        # 去除多余的批次维度添加
        batch_reshaped = batch.reshape((batch_size, 28, 28, 1))  # 保持数据的秩为 4

        try:
            batch_predictions = mlmodel.predict({"conv2d_input": batch_reshaped})
            predictions.append(batch_predictions)
        except Exception as e:
            print(f"Error during batch inference {batch_idx}: {e}")

    end_time_inference = time.time()
    inference_duration = end_time_inference - start_time_inference
    print(f'Time taken for inference on {len(X_test_sample)} samples: {inference_duration:.4f} seconds')

    stop_event.set()  # 触发停止其他线程



# 运行外部脚本并捕获输出
def run_script(stop_event):
    try:
        # 使用临时文件来存储子进程输出
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            process = subprocess.Popen(['/Users/anelloyi/Desktop/run_powermetrics.sh'], 
                                       stdout=tmp_file, stderr=subprocess.STDOUT, text=True)
            print("Subprocess started.")
            while not stop_event.is_set():
                if process.poll() is not None:  # 检查进程是否已经结束
                    break
            
            if process.poll() is None:
                # 尝试终止进程
                process.terminate()
                try:
                    process.wait(timeout=0.1)
                except subprocess.TimeoutExpired:
                    process.kill()  # 如果超时，则强制终止
                    process.wait()
        
        # 读取临时文件的内容
        with open(tmp_file.name, 'r') as f:
            thread_output['powermetrics'] = f.read()
        
        os.remove(tmp_file.name)  # 删除临时文件
        print("Subprocess finished.")
    except Exception as e:
        thread_output['powermetrics'] = str(e)
        print("Exception in subprocess:", str(e))


stop_event = threading.Event()
# model = tf.keras.models.load_model("./mnist_models/mnist_cnn_model.h5")
model = tf.keras.models.load_model("./models_train/mnist_cnn_model.h5")


# 创建和启动线程
thread1 = threading.Thread(target=save_model, args=(stop_event, model))
thread2 = threading.Thread(target=monitor_resources_during_save, args=(stop_event,))
thread3 = threading.Thread(target=run_script, args=(stop_event,))

thread3.start()
thread1.start()
thread2.start()

# 等待线程完成
thread1.join()
thread2.join()
thread3.join()

# 输出从线程收集的数据
content = thread_output.get('powermetrics', 'No output captured')
# 将内容按行拆分
lines = content.split('\n')
# 筛选出以 "CPU consume" 开头的行
filtered_lines = [line for line in lines if line.startswith('GPU Power:') or 
                  line.startswith('CPU Power:')]
# 将筛选后的行合并为一个字符串，每行之间用换行符分隔
filtered_content = '\n'.join(filtered_lines)

# ----------------------------------------------------------------------------
file_name = f'./mnist_models/output_cnn/output-mnist-cnn-coreml.txt'
# ----------------------------------------------------------------------------
with open(file_name, 'w') as file:
    file.write(filtered_content)
    file.write(f'\nTotal Duration(s): {duration:.2f}')
print(f"Content saved to {file_name}")

filtered_lines_count = len(filtered_lines)
# print(filtered_lines_count)
# print(filtered_lines)

duration = inference_duration
# 提取每一个采样点的数字，即CPU和GPU的具体mV
numbers = []
for line in filtered_lines:
    match = re.search(r'[\d.]+', line)
    if match:
        numbers.append(float(match.group())) 
# print("Extracted numbers:", numbers)

delta_time = duration * 2 / filtered_lines_count
# print(delta_time)

numbers_scaled = [num * delta_time for num in numbers]
total_energy_consumption = sum(numbers_scaled)
print(f"Total energy consumption: {total_energy_consumption} mV")


When both 'convert_to' and 'minimum_deployment_target' not specified, 'convert_to' is set to "mlprogram" and 'minimum_deployment_target' is set to ct.target.iOS15 (which is same as ct.target.macOS12). Note: the model will not run on systems older than iOS15/macOS12/watchOS8/tvOS15. In order to make your model run on older system, please set the 'minimum_deployment_target' to iOS14/iOS13. Details please see the link: https://apple.github.io/coremltools/docs-guides/source/target-conversion-formats.html


Saving model...
Subprocess started.


Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 1697.07 passes/s]
When both 'convert_to' and 'minimum_deployment_target' not specified, 'convert_to' is set to "mlprogram" and 'minimum_deployment_target' is set to ct.target.iOS15 (which is same as ct.target.macOS12). Note: the model will not run on systems older than iOS15/macOS12/watchOS8/tvOS15. In order to make your model run on older system, please set the 'minimum_deployment_target' to iOS14/iOS13. Details please see the link: https://apple.github.io/coremltools/docs-guides/source/target-conversion-formats.html
Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 1569.68 passes/s]
When both 'convert_to' and 'minimum_deployment_target' not specified, 'convert_to' is set to "mlprogram" and 'minimum_deployment_target' is set to ct.target.iOS15 (which is same as ct.target.macOS12). Note: the model will not run on systems older than iOS15/macOS12/watchOS8/tvOS15. In order to make your

Time taken to save Core ML model: 6.624547719955444 seconds
Time taken for inference on 10000 samples: 4.8873 seconds
Subprocess finished.
Resource monitoring finished.
Content saved to ./mnist_models/output_cnn/output-mnist-cnn-coreml.txt
Total energy consumption: 36148.15653216859 mV


In [10]:
import threading
import time
import numpy as np
import psutil
import subprocess
import os
import tempfile
import tensorflow as tf
import coremltools as ct
import re

thread_output = {}
duration = 0
inference_duration = 0  # 添加推理时间变量

def monitor_resources_during_save(stop_event):
    cpu_usage = []
    gpu_usage = []
    while not stop_event.is_set():
        cpu_usage.append(psutil.cpu_percent(interval=0.1))
        try:
            gpu_output = subprocess.check_output(['nvidia-smi', '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
            gpu_usage.append(int(gpu_output.strip()))
        except Exception as e:
            gpu_usage.append(None)  # 如果没有GPU或nvidia-smi命令失败，则记录None
    thread_output['cpu_usage'] = cpu_usage
    thread_output['gpu_usage'] = gpu_usage
    print("Resource monitoring finished.")

def save_model(stop_event, model, batch_size=128):
    global duration
    global inference_duration
    print("Saving model...")
    start_time = time.time()
    try:
        input_shape = ct.Shape(shape=(1, 28, 28, 1))
        input_feature = ct.TensorType(name="conv2d_input", shape=input_shape)
        mlmodel = ct.convert(model, inputs=[input_feature])
        mlmodel.save(f"mnist_cnn_model_coreML.mlpackage")
    except Exception as e:
        print(f"Error saving model: {e}")
    end_time = time.time()
    duration = end_time - start_time
    print(f'Time taken to save Core ML model: {duration} seconds')

    # 测量推理时间
    X_test_sample = X_test[:10000].astype(np.float32)  # 选择前1000个样本进行推理
    start_time_inference = time.time()
    predictions = []
    for idx in range(len(X_test_sample)):
        batch = X_test_sample[idx]
        batch_reshaped = batch.reshape((1, 28, 28, 1))  # 一次只处理一个样本，匹配模型输入维度
        try:
            batch_predictions = mlmodel.predict({"conv2d_input": batch_reshaped})
            predictions.append(batch_predictions)
        except Exception as e:
            print(f"Error during inference for sample {idx}: {e}")
    end_time_inference = time.time()
    inference_duration = end_time_inference - start_time_inference
    print(f'Time taken for inference on {len(X_test_sample)} samples: {inference_duration:.4f} seconds')

    stop_event.set()


def run_script(stop_event):
    try:
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            process = subprocess.Popen(['/Users/anelloyi/Desktop/run_powermetrics.sh'], stdout=tmp_file, stderr=subprocess.STDOUT, text=True)
            print("Subprocess started.")
            while not stop_event.is_set():
                if process.poll() is not None:
                    break
            if process.poll() is None:
                process.terminate()
                try:
                    process.wait(timeout=0.1)
                except subprocess.TimeoutExpired:
                    process.kill()
                    process.wait()
        with open(tmp_file.name, 'r') as f:
            thread_output['powermetrics'] = f.read()
        os.remove(tmp_file.name)
        print("Subprocess finished.")
    except Exception as e:
        thread_output['powermetrics'] = str(e)
        print("Exception in subprocess:", str(e))

stop_event = threading.Event()
model = tf.keras.models.load_model("./models_train/mnist_cnn_model.h5")

thread1 = threading.Thread(target=save_model, args=(stop_event, model, 128))  # 可修改批量大小为 128 或其他值
thread2 = threading.Thread(target=monitor_resources_during_save, args=(stop_event,))
thread3 = threading.Thread(target=run_script, args=(stop_event,))

thread1.start()
thread2.start()
thread3.start()
thread1.join()
thread2.join()
thread3.join()

content = thread_output.get('powermetrics', 'No output captured')
lines = content.split('\n')
filtered_lines = [line for line in lines if line.startswith('GPU Power:') or line.startswith('CPU Power:')]
filtered_content = '\n'.join(filtered_lines)

file_name = f'./mnist_models/output_cnn/output-mnist-cnn-coreml.txt'
with open(file_name, 'w') as file:
    file.write(filtered_content)
    file.write(f'\nTotal Duration(s): {duration:.2f}')
print(f"Content saved to {file_name}")

filtered_lines_count = len(filtered_lines)
numbers = []
for line in filtered_lines:
    match = re.search(r'[\d.]+', line)
    if match:
        numbers.append(float(match.group()))
delta_time = duration * 2 / filtered_lines_count
numbers_scaled = [num * delta_time for num in numbers]
total_energy_consumption = sum(numbers_scaled)
print(f"Total energy consumption: {total_energy_consumption} mV")


When both 'convert_to' and 'minimum_deployment_target' not specified, 'convert_to' is set to "mlprogram" and 'minimum_deployment_target' is set to ct.target.iOS15 (which is same as ct.target.macOS12). Note: the model will not run on systems older than iOS15/macOS12/watchOS8/tvOS15. In order to make your model run on older system, please set the 'minimum_deployment_target' to iOS14/iOS13. Details please see the link: https://apple.github.io/coremltools/docs-guides/source/target-conversion-formats.html


Saving model...
Subprocess started.


Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 1701.89 passes/s]


Time taken to save Core ML model: 1.6163678169250488 seconds
Time taken for inference on 10000 samples: 4.9429 seconds
Subprocess finished.
Resource monitoring finished.
Content saved to ./mnist_models/output_cnn/output-mnist-cnn-coreml.txt
Total energy consumption: 14268.401959651219 mV


### CNN pytorch

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import numpy as np

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64*5*5, 128)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 10)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# pytorch_model = CNNModel()

import torch
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset

# 设置要选择的测试集数量
test_sample_size = 10000  # 例如，选择10000个测试样本

# 加载MNIST测试数据
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

test_dataset = datasets.MNIST('.', train=False, transform=transform, download=True)

# 从测试集中选择指定数量的样本
indices = list(range(len(test_dataset)))
np.random.shuffle(indices)
selected_indices = indices[:test_sample_size]
subset_test_dataset = Subset(test_dataset, selected_indices)

test_loader = DataLoader(subset_test_dataset, batch_size=128, shuffle=False)  # 使用更大的batch_size进行批量推理


In [9]:
import time
import threading
import psutil
import subprocess
import tempfile
import os
import re
import tensorflow as tf
import torch

thread_output = {}
duration = 0
inference_duration = 0  # 添加用于存储推理时间的变量

# 定义监控保存模型时的资源使用率的线程函数
def monitor_resources_during_save(stop_event):
    cpu_usage = []
    gpu_usage = []

    while not stop_event.is_set():
        cpu_usage.append(psutil.cpu_percent(interval=0.1))
        try:
            gpu_output = subprocess.check_output(['nvidia-smi', 
                                                  '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
            gpu_usage.append(int(gpu_output.strip()))
        except Exception as e:
            gpu_usage.append(None)  # 如果没有GPU或nvidia-smi命令失败，则记录None

    # 保存监测结果
    thread_output['cpu_usage'] = cpu_usage
    thread_output['gpu_usage'] = gpu_usage
    print("Resource monitoring finished.")

# 定义保存模型的函数
def save_model(stop_event, model):
    global duration
    global inference_duration
    print("Saving model...")
    start_time = time.time()

    # 提取权重并分别保存为.npy文件
    for i, layer in enumerate(model.layers):
        weights = layer.get_weights()
        for j, weight in enumerate(weights):
            np.save(f'layer_{i}_weight_{j}.npy', weight)

    pytorch_model = CNNModel()

    # 为PyTorch模型赋值
    with torch.no_grad():
        pytorch_model.conv1.weight = nn.Parameter(torch.tensor(np.load('layer_0_weight_0.npy')).permute(3, 2, 0, 1))
        pytorch_model.conv1.bias = nn.Parameter(torch.tensor(np.load('layer_0_weight_1.npy')))

        pytorch_model.conv2.weight = nn.Parameter(torch.tensor(np.load('layer_2_weight_0.npy')).permute(3, 2, 0, 1))
        pytorch_model.conv2.bias = nn.Parameter(torch.tensor(np.load('layer_2_weight_1.npy')))

        pytorch_model.fc1.weight = nn.Parameter(torch.tensor(np.load('layer_5_weight_0.npy').T))
        pytorch_model.fc1.bias = nn.Parameter(torch.tensor(np.load('layer_5_weight_1.npy')))

        pytorch_model.fc2.weight = nn.Parameter(torch.tensor(np.load('layer_7_weight_0.npy').T))
        pytorch_model.fc2.bias = nn.Parameter(torch.tensor(np.load('layer_7_weight_1.npy')))

    # 保存整个模型
    for i in range(1):
        torch.save(pytorch_model, 'mnist_cnn_model.pth')


    end_time = time.time()
    duration = end_time - start_time
    print(f'Time taken to save PyTorch model: {duration} seconds')

    #################

    # 测量推理时间
    start_time = time.time()

    # 切换模型到评估模式
    pytorch_model.eval()

    n_time = 1
    for i in range(n_time):
        # 进行推理
        correct = 0
        with torch.no_grad():
            for data, target in test_loader:
                output = pytorch_model(data)
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()

        print(f'Test Accuracy on {test_sample_size} samples: {correct / test_sample_size * 100:.2f}%')


    end_time = time.time()
    inference_duration = end_time - start_time
    print(f'Time taken for inference on {test_sample_size} samples, {n_time} times: {inference_duration:.4f} seconds')

    stop_event.set()  # 触发停止其他线程

# 运行外部脚本并捕获输出
def run_script(stop_event):
    try:
        # 使用临时文件来存储子进程输出
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            process = subprocess.Popen(['/Users/anelloyi/Desktop/run_powermetrics.sh'], 
                                       stdout=tmp_file, stderr=subprocess.STDOUT, text=True)
            print("Subprocess started.")
            while not stop_event.is_set():
                if process.poll() is not None:  # 检查进程是否已经结束
                    break
            
            if process.poll() is None:
                # 尝试终止进程
                process.terminate()
                try:
                    process.wait(timeout=0.1)
                except subprocess.TimeoutExpired:
                    process.kill()  # 如果超时，则强制终止
                    process.wait()
        
        # 读取临时文件的内容
        with open(tmp_file.name, 'r') as f:
            thread_output['powermetrics'] = f.read()
        
        os.remove(tmp_file.name)  # 删除临时文件
        print("Subprocess finished.")
    except Exception as e:
        thread_output['powermetrics'] = str(e)
        print("Exception in subprocess:", str(e))

# 定义推理测试的线程函数
def test_inference(stop_event, model):
    global inference_duration
    print("Testing inference...")
    start_time = time.time()

    # 示例推理代码：假设输入数据为随机生成的
    input_data = tf.random.normal([1, 28, 28, 1])  # 根据你的模型调整输入形状
    model.predict(input_data)

    end_time = time.time()
    inference_duration = end_time - start_time
    print(f'Time taken for inference: {inference_duration} seconds')
    stop_event.set()  # 触发停止其他线程

stop_event = threading.Event()
model = tf.keras.models.load_model("./mnist_models/mnist_cnn_model.h5")

# 创建和启动线程
thread1 = threading.Thread(target=save_model, args=(stop_event, model))
thread2 = threading.Thread(target=monitor_resources_during_save, args=(stop_event,))
thread3 = threading.Thread(target=run_script, args=(stop_event,))
# thread4 = threading.Thread(target=test_inference, args=(stop_event, model))

thread3.start()
thread1.start()
thread2.start()

# 等待线程完成
thread1.join()
thread2.join()
thread3.join()

# 最后再做推理
# thread4.start()
# thread4.join()

# 输出从线程收集的数据
content = thread_output.get('powermetrics', 'No output captured')
lines = content.split('\n')
filtered_lines = [line for line in lines if line.startswith('GPU Power:') or 
                  line.startswith('CPU Power:')]

filtered_content = '\n'.join(filtered_lines)

# ----------------------------------------------------------------------------
file_name = f'./mnist_models/output_cnn/output-mnist-cnn-pth.txt'
# ----------------------------------------------------------------------------
with open(file_name, 'w') as file:
    file.write(filtered_content)
    file.write(f'\nTotal Duration(s): {duration:.2f}')
    file.write(f'\nInference Duration(s): {inference_duration:.2f}')
print(f"Content saved to {file_name}")

filtered_lines_count = len(filtered_lines)
#print(filtered_lines_count)
#print(filtered_lines)

duration = inference_duration
# 确保 filtered_lines_count 不为零
if filtered_lines_count > 0:
    # 提取每一个采样点的数字，即CPU和GPU的具体mV
    numbers = []
    for line in filtered_lines:
        match = re.search(r'[\d.]+', line)
        if match:
            numbers.append(float(match.group()))

    delta_time = duration * 2 / filtered_lines_count
    numbers_scaled = [num * delta_time for num in numbers]
    total_energy_consumption = sum(numbers_scaled)
    print(f"Total energy consumption: {total_energy_consumption} mV")
else:
    print("No filtered lines to process.")


Saving model...
Subprocess started.
Time taken to save PyTorch model: 0.039903879165649414 seconds
Test Accuracy on 10000 samples: 15.97%
Time taken for inference on 10000 samples, 1 times: 6.0333 seconds
Subprocess finished.
Resource monitoring finished.
Content saved to ./mnist_models/output_cnn/output-mnist-cnn-pth.txt
Total energy consumption: 40215.5893334991 mV
